In [1]:
import sys
sys.path.append('C:\My_files\Programming\work')
import pandas as pd
from processing import YouScanProcessing as ysp
import numpy as np
import re
from gspread_pandas import Spread, Client
import gspread_pandas
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [2]:
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('C:/My_files/Programming/work/google_secret.json', scope)
gc = gspread.authorize(credentials)

In [3]:
def del_tag_num(name):
    return re.sub('\d{1}_', '', name)

In [4]:
def sort_dates(data):
    data.index = pd.to_datetime(data.index, dayfirst=True)
    data.sort_index(inplace=True)
    data.index = data.index.strftime('%d.%m.%Y')

In [5]:
sht = gc.open('geely_atlas_pro')

In [6]:
w = [int(i) for i in sht.get_worksheet(0).acell('D44').value.split(' ')]
m = [int(i) for i in sht.get_worksheet(0).acell('H26').value.split(' ')]
# m1 = [int(i) for i in sht.get_worksheet(0).acell('H22').value.split(' ')]

In [7]:
m,w

([601, 382], [164, 50])

In [57]:
sht.get_worksheet(0).update('E27', f'{w[0]} {w[1]}')

{'spreadsheetId': '149Qd6JED8E4LCZoU7a5PfQ8A2ffPogz--fEAHjTcNQk',
 'updatedRange': 'Data!E27',
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [8]:
df = pd.read_excel('atlas_pro24.xlsx')

In [9]:
df['week'] = df['Дата'].apply(lambda x: pd.to_datetime(x, dayfirst=True) \
                       .isocalendar().week).to_list()
df['month'] = df['Дата'].apply(lambda x: pd.to_datetime(x, dayfirst=True) \
                        .month).to_list()

# SLIDE 1

In [11]:
all_time = [int(sht.get_worksheet(0).acell('J16').value),
int(sht.get_worksheet(0).acell('K16').value),
int(sht.get_worksheet(0).acell('L16').value),
int(sht.get_worksheet(0).acell('M16').value)]

In [12]:
all_time

[17062, 339987, 226996166, 54860596]

In [13]:
temp_dict = {
    'Posts': df.shape[0],
    'Engagements': df[['Репосты','Комментарии','Сумма всех реакций']].sum().sum(),
    'OTS_': df.groupby('Профиль места публикации').agg({'Подписчики места публикации': 'mean'}).sum()[0],
    'OTS': df.groupby('Профиль').agg({'Подписчики': 'mean'}).sum()[0]   
}

main = pd.DataFrame(temp_dict, index=['week']) \
         .transpose().astype(int)
main['week_R'] = main.week.apply(ysp.rounding)
main['all_time'] = all_time + main.week
main['all_time_R'] = main.all_time.apply(ysp.rounding)

main.transpose()

,Posts,Engagements,OTS_,OTS
week,434,3848,16759139,2351119
week_R,434,3.8K,16.8M,2.4M
all_time,17496,343835,243755305,57211715
all_time_R,17.5K,343.8K,243.8M,57.2M


# SLIDE 2

In [14]:
count_week = df.groupby('week').agg({'Дата': 'count'}).rename(columns={'Дата': 'amount'})
count_month = df.groupby('month').agg({'Дата': 'count'}).rename(columns={'Дата': 'amount'})

In [16]:
count_week
# count_month

,amount
week,
51,130
52,304


In [17]:
weeks = pd.concat([
    df[(df['1_Отзыв'] == '1_Отзыв')].groupby('week').agg({'Дата': 'count'}).rename(columns={'Дата': 'reviews'}),
    df[(df['1_Новость/Статья'] == '1_Новость/Статья')].groupby('week').agg({'Дата': 'count'}).rename(columns={'Дата': 'news'})], axis=1)
weeks.loc[weeks.index[-2], 'reviews'] = weeks.loc[weeks.index[-2], 'reviews'] + w[0]
weeks.loc[weeks.index[-2], 'news'] = weeks.loc[weeks.index[-2], 'news'] + w[1]
weeks['reviews%'] = (weeks.reviews / (weeks.reviews + weeks.news)).round(3)
weeks['news%'] = (weeks.news / (weeks.reviews + weeks.news)).round(3)

In [18]:
weeks

,reviews,news,reviews%,news%
week,,,,
51,243,73,0.769,0.231
52,69,201,0.256,0.744


In [19]:
monthes = pd.concat([
    df[(df['1_Отзыв'] == '1_Отзыв')].groupby('month').agg({'Дата': 'count'}).rename(columns={'Дата': 'reviews'})+m[0],
    df[(df['1_Новость/Статья'] == '1_Новость/Статья')].groupby('month').agg({'Дата': 'count'}).rename(columns={'Дата': 'news'})+m[1]
], axis=1)
monthes['reviews%'] = ((monthes.reviews) / (monthes.reviews + monthes.news)).round(3)
monthes['news%'] = ((monthes.news) / (monthes.reviews + monthes.news)).round(3)

In [20]:
monthes

,reviews,news,reviews%,news%
month,,,,
12,749,606,0.553,0.447


# SLIDE 3 sourse types

In [21]:
source_types = df.groupby('Тип источника') \
             .agg({'Тип источника': 'count'}) \
             .rename(columns={'Тип источника':'count'})
source_types

,count
Тип источника,
Блог,31
Мессенджеры,66
Отзывы,1
СМИ,29
Соц. сеть,300
Форум,7


In [22]:
source_types = df.groupby('Тип источника') \
             .agg({'Тип источника': 'count'}) \
             .rename(columns={'Тип источника':'count'})
source_types.at['Блог и форумы'] = source_types.loc['Блог'] + source_types.loc['Форум']
try:
    source_types = source_types.iloc[np.r_[1:5, 6:7]].sort_index().astype(int)
except:
    source_types = source_types.iloc[np.r_[1:4, 5:6]].sort_index().astype(int)
in_perc = [(f'{source_types.loc[index][0].astype(int)} ({(source_types.loc[index][0] / source_types.sum()[0] * 100).round().astype(int)}%)') for index in source_types.index]
source_types['in %'] = in_perc

source_types

,count,in %
Тип источника,,
Блог и форумы,38,38 (9%)
Мессенджеры,66,66 (15%)
Отзывы,1,1 (0%)
СМИ,29,29 (7%)
Соц. сеть,300,300 (69%)


# SLIDE 3 sourses

In [23]:
source = df.groupby('Источник') \
             .agg({'Источник': 'count'}) \
             .rename(columns={'Источник': 'scount'}) \
             .sort_values('scount', ascending=False)[:10]
source = source \
        .append(pd.DataFrame([df['Источник'].nunique()], 
                             columns=['scount'])) \
        .rename({0: 'Unique source'})

source

,scount
vk.com,155
telegram.me,86
youtube.com,64
instagram.com,31
facebook.com,22
ok.ru,14
twitter.com,13
drive2.ru,7
zen.yandex.ru,5
findjob.ru,3


# SLIDE 5 total tonality

In [24]:
total_tonality = ysp.nsi(df)
total_tonality.rename(columns={'Негативная': 'Negative', 
                    'Нейтральная': 'Neutral', 
                    'Позитивная': 'Positive', 
                    'Итого': 'Total',}, inplace=True)
total_tonality = total_tonality.loc['Итого'].to_frame()[:-1].astype(int)
total_tonality

,Итого
Negative,6
Neutral,420
Positive,8
Total,434


# SLIDE 5 tags tonality

In [25]:
tags = pd.DataFrame()
for col in df.columns:
        if str(col).startswith('2_'):
            neg = df[(df['Тональность'] == 'Негативная')][col].count()
            neu = df[(df['Тональность'] == 'Нейтральная')][col].count()
            pos = df[(df['Тональность'] == 'Позитивная')][col].count()
            temp = pd.DataFrame({col: {'Negative': neg, 
                                        'Neutral': neu, 
                                        'Positive': pos, 
                                        'Total': sum([neg, neu, pos])}}).transpose()

            tags = tags.append(temp).sort_values('Total', ascending=False).rename(index=del_tag_num).astype(int)
tags = ysp.percent(tags,
                   div_round=3, to_percent=False,
                   calc_columns=tags.columns,
                   total_types=False, 
                   percent_columns=[f'{col} %' for col in tags.columns])
tags

,Negative,Neutral,Positive,Total,Negative %,Neutral %,Positive %
Service,1,65,0,66,0.015,0.985,0.000
Package,1,34,0,35,0.029,0.971,0.000
Quality,2,30,2,34,0.059,0.882,0.059
Price,3,10,0,13,0.231,0.769,0.000
Comparison,1,10,1,12,0.083,0.833,0.083
Design,1,6,0,7,0.143,0.857,0.000
Brand,0,1,0,1,0.000,1.000,0.000
Rust,0,0,0,0,NaN,NaN,NaN
Suspension,0,0,0,0,NaN,NaN,NaN


In [23]:
pos_links = ( 
    df[(df['Тип поста'] == 'Пост') & 
    (df['Тональность'] == 'Позитивная')]
    .groupby('URL')
    .agg({'Дата': 'count'})
    .sort_values('Дата', ascending=False)
    .reset_index()
    )
# pos_links[pos_links.URL.str.contains('youtube.com')]
# pos_links[pos_links.URL.str.contains('zen.yandex.ru')]
# pos_links[pos_links.URL.str.contains('auto.mail.ru')]

In [24]:
neu_links = ( 
    df[(df['Тип поста'] == 'Пост') & 
    (df['Тональность'] == 'Нейтральная')]
    .groupby('URL')
    .agg({'Дата': 'count'})
    .sort_values('Дата', ascending=False)
    .reset_index()
    )
# neu_links[neu_links.URL.str.contains('youtube.com')]
neu_links[neu_links.URL.str.contains('zen.yandex.ru')]
# neu_links[neu_links.URL.str.contains('auto.mail.ru')]

,URL,Дата
81,https://zen.yandex.ru/media/id/5932b5028e557d3...,1
82,https://zen.yandex.ru/media/id/5c45963a3a08b80...,1
83,https://zen.yandex.ru/media/id/5e8b0557c69c707...,1
84,https://zen.yandex.ru/media/id/5f283cfb2d6c3a7...,1
85,https://zen.yandex.ru/media/id/5fc5ee1b125e451...,1
228,https://zen.yandex.ru/media/id/5ffadce91660cc5...,1


In [25]:
neg_links = ( 
    df[(df['Тип поста'] == 'Пост') & 
    (df['Тональность'] == 'Негативная')]
    .groupby('URL')
    .agg({'Дата': 'count'})
    .sort_values('Дата', ascending=False)
    .reset_index()
    )   
# neg_links[neg_links.URL.str.contains('youtube.com')]
neg_links[neg_links.URL.str.contains('zen.yandex.ru')]
# neg_links[neg_links.URL.str.contains('auto.mail.ru')]

,URL,Дата


# UPLOAD DATA

In [26]:
c = gspread_pandas.conf.get_config(conf_dir='C:\My_files\Programming\work', file_name='google_secret.json')
spread = Spread('geely_atlas_pro', config=c)
spread.sheets

[<Worksheet 'Data' id:1045000603>,
 <Worksheet 'links' id:532102312>,
 <Worksheet 'Charts' id:665156519>]

In [27]:
# НЕ ЗАБЫВАТЬ двигать на нужные строки!!!
# spread.df_to_sheet(weeks[['reviews%','news%']], sheet='Data', start='A23', headers=False)
# spread.df_to_sheet(monthes[['reviews%','news%']], sheet='Data', start='E20', headers=False)
# spread.df_to_sheet(count_week, sheet='Data', start='I23', headers=False)
# spread.df_to_sheet(count_month, sheet='Data', start='L20', headers=False)

In [27]:
spread.df_to_sheet(source_types, sheet='Data', start='O2')
spread.df_to_sheet(source, sheet='Data', start='O9')

In [28]:
spread.df_to_sheet(tags, sheet='Data', start='A2')
spread.df_to_sheet(total_tonality, sheet='Data', start='J2')
spread.df_to_sheet(main.transpose(), sheet='Data', start='I13')